In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds 

In [ ]:
data, metadata = tfds.load("mnist", as_supervised=True, with_info=True)

In [ ]:
metadata

In [ ]:
train_data, test_data = data['train'], data['test']
train_data

In [ ]:
labels = metadata.features['label'].names
labels

In [ ]:
#normalizing info
def normalize(images, labels):
  images = tf.cast(images, tf.float32)
  images /= 255
  return images, labels

train_data = train_data.map(normalize)
test_data = test_data.map(normalize)

train_data = train_data.cache()
test_data = test_data.cache()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
for i, (image, label) in enumerate(train_data.take(25)):
  image = image.numpy().reshape((28,28))
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image,cmap=plt.cm.binary)
  plt.xlabel(labels[label])
plt.show()

In [ ]:
# creacion del modelo
model = tf.keras.Sequential([

  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),

  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),

  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),

  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Flatten(),

  tf.keras.layers.Dense(30, activation=tf.nn.relu),
  tf.keras.layers.Dense(30, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [ ]:
model.compile(
    optimizer='adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

In [ ]:
num_train_data = metadata.splits['train'].num_examples
num_test_data = metadata.splits['test'].num_examples

In [ ]:
TRAIN_BATCH = 30

train_data = train_data.repeat().shuffle(60000).batch(TRAIN_BATCH)
test_data = test_data.batch(TRAIN_BATCH)

In [ ]:
import math
## training
trainig = model.fit(train_data, epochs=5, steps_per_epoch=math.ceil(num_train_data/TRAIN_BATCH))

In [ ]:
plt.xlabel("# Epoca")
plt.ylabel("Pérdida")
plt.plot(trainig.history["loss"])

In [ ]:
model.save('model.h5')

In [ ]:
!pip install tensorflowjs

In [ ]:
!mkdir carpeta_salida

In [ ]:
!tensorflowjs_converter --input_format keras model.h5 carpeta_salida